# Breast cancer relapse classification 

In this IPython notebook we folmulate breast cancer relapse prediction as a classification problem. We use the PFSNet based scores to construct features for the classfier. We then perform predictions of the cancer relapses.

In [1]:
#Import common Python modules
import numpy as np
import scipy as sp
import matplotlib as plot
import Bio
import pandas as pd

%matplotlib inline


In [2]:
#Load the datasets into pandas dataframes.
dataset_column = open("datasets/breast_cancer_dataset_columns.txt","r")
dataset = pd.read_csv('datasets/breast_cancer_dataset.txt', delimiter='\t', usecols=dataset_column.read().split('\t'))
relapse_dataset = pd.read_csv('datasets/relapse_info.txt', delimiter='\t')
id_conversion = pd.read_csv('id_conversion_data.txt', delimiter='\t')

## 1. Initial data preprocessing

In [57]:
#Create a file with all the affymetrix gene names
f = open("affymetrix_genes.txt","w+")
for index in dataset.index:
    f.write(str(index)+"\n")
f.close()

In [74]:
#Create a file with the standard gene names. Unfortunatelly, using online tool it was not possible to resolve all the names.
g = open("gene_id.txt", "w+")
gene_names = []
for gene_name in id_conversion['Gene Name']:
    gene = gene_name.split(' ')[-1].split('(')[-1][:-1]
    gene_names.append(gene)
    g.write(str(gene)+"\n")
g.close()

In [125]:
# Create a gene correspondens table
affymetrix_gene_conversion_table = {}
for gene_id in id_conversion['ID']:
    for g_id in gene_id.split(','):
        affymetrix_gene_conversion_table[g_id] = str(id_conversion[id_conversion['ID'] == gene_id]['Gene Name'].values[0].split(' ')[-1].split('(')[-1][:-1])                                                       

In [384]:
#Using current method it was not possible to find gene names for most of the elements. 
#Therefore, we decide to use an alternative approach.
i = 1
for index in dataset.index:
    if index in affymetrix_gene_conversion_table.keys():
        print(str(i) +". "+ str(index)+": "+str(affymetrix_gene_conversion_table[index]))
        i = i + 1
    else:
        print("Failed to find the corresponding gene name.")

Failed to find the corresponding gene name.
Failed to find the corresponding gene name.
Failed to find the corresponding gene name.
Failed to find the corresponding gene name.
Failed to find the corresponding gene name.
Failed to find the corresponding gene name.
Failed to find the corresponding gene name.
Failed to find the corresponding gene name.
Failed to find the corresponding gene name.
Failed to find the corresponding gene name.
1. 1438_at: EPHB3
Failed to find the corresponding gene name.
Failed to find the corresponding gene name.
Failed to find the corresponding gene name.
Failed to find the corresponding gene name.
Failed to find the corresponding gene name.
2. 1773_at: CHURC1-FNTB
Failed to find the corresponding gene name.
Failed to find the corresponding gene name.
3. 1861_at: BAD
Failed to find the corresponding gene name.
Failed to find the corresponding gene name.
Failed to find the corresponding gene name.
Failed to find the corresponding gene name.
Failed to find the

Failed to find the corresponding gene name.
31. 200860_s_at: CNOT1
Failed to find the corresponding gene name.
32. 200862_at: DHCR24
Failed to find the corresponding gene name.
Failed to find the corresponding gene name.
Failed to find the corresponding gene name.
Failed to find the corresponding gene name.
Failed to find the corresponding gene name.
Failed to find the corresponding gene name.
Failed to find the corresponding gene name.
Failed to find the corresponding gene name.
Failed to find the corresponding gene name.
Failed to find the corresponding gene name.
Failed to find the corresponding gene name.
Failed to find the corresponding gene name.
Failed to find the corresponding gene name.
Failed to find the corresponding gene name.
Failed to find the corresponding gene name.
Failed to find the corresponding gene name.
Failed to find the corresponding gene name.
33. 200880_at: DNAJA1
Failed to find the corresponding gene name.
Failed to find the corresponding gene name.
Failed to

Failed to find the corresponding gene name.
Failed to find the corresponding gene name.
Failed to find the corresponding gene name.
Failed to find the corresponding gene name.
Failed to find the corresponding gene name.
Failed to find the corresponding gene name.
Failed to find the corresponding gene name.
Failed to find the corresponding gene name.
59. 201253_s_at: CDIPT
Failed to find the corresponding gene name.
60. 201255_x_at: BAG6
Failed to find the corresponding gene name.
Failed to find the corresponding gene name.
Failed to find the corresponding gene name.
Failed to find the corresponding gene name.
Failed to find the corresponding gene name.
Failed to find the corresponding gene name.
Failed to find the corresponding gene name.
Failed to find the corresponding gene name.
Failed to find the corresponding gene name.
Failed to find the corresponding gene name.
Failed to find the corresponding gene name.
Failed to find the corresponding gene name.
Failed to find the correspondin

Failed to find the corresponding gene name.
Failed to find the corresponding gene name.
Failed to find the corresponding gene name.
Failed to find the corresponding gene name.
Failed to find the corresponding gene name.
Failed to find the corresponding gene name.
Failed to find the corresponding gene name.
86. 201637_s_at: FXR1
Failed to find the corresponding gene name.
Failed to find the corresponding gene name.
87. 201640_x_at: CLPTM1
Failed to find the corresponding gene name.
Failed to find the corresponding gene name.
Failed to find the corresponding gene name.
Failed to find the corresponding gene name.
Failed to find the corresponding gene name.
Failed to find the corresponding gene name.
Failed to find the corresponding gene name.
88. 201648_at: JAK1
Failed to find the corresponding gene name.
Failed to find the corresponding gene name.
Failed to find the corresponding gene name.
89. 201652_at: COPS5
Failed to find the corresponding gene name.
Failed to find the corresponding 

Failed to find the corresponding gene name.
Failed to find the corresponding gene name.
Failed to find the corresponding gene name.
Failed to find the corresponding gene name.
Failed to find the corresponding gene name.
Failed to find the corresponding gene name.
Failed to find the corresponding gene name.
Failed to find the corresponding gene name.
Failed to find the corresponding gene name.
Failed to find the corresponding gene name.
Failed to find the corresponding gene name.
Failed to find the corresponding gene name.
Failed to find the corresponding gene name.
Failed to find the corresponding gene name.
Failed to find the corresponding gene name.
Failed to find the corresponding gene name.
Failed to find the corresponding gene name.
Failed to find the corresponding gene name.
Failed to find the corresponding gene name.
129. 202041_s_at: FIBP
Failed to find the corresponding gene name.
Failed to find the corresponding gene name.
Failed to find the corresponding gene name.
Failed to

...

Failed to find the corresponding gene name.
Failed to find the corresponding gene name.
Failed to find the corresponding gene name.
Failed to find the corresponding gene name.
Failed to find the corresponding gene name.
Failed to find the corresponding gene name.
Failed to find the corresponding gene name.
Failed to find the corresponding gene name.
Failed to find the corresponding gene name.
Failed to find the corresponding gene name.
Failed to find the corresponding gene name.
Failed to find the corresponding gene name.
Failed to find the corresponding gene name.
Failed to find the corresponding gene name.
Failed to find the corresponding gene name.
Failed to find the corresponding gene name.
Failed to find the corresponding gene name.
Failed to find the corresponding gene name.
Failed to find the corresponding gene name.
Failed to find the corresponding gene name.
Failed to find the corresponding gene name.
Failed to find the corresponding gene name.
Failed to find the corresponding

Failed to find the corresponding gene name.
Failed to find the corresponding gene name.
Failed to find the corresponding gene name.
Failed to find the corresponding gene name.
Failed to find the corresponding gene name.
Failed to find the corresponding gene name.
Failed to find the corresponding gene name.
Failed to find the corresponding gene name.
Failed to find the corresponding gene name.
Failed to find the corresponding gene name.
Failed to find the corresponding gene name.
Failed to find the corresponding gene name.
Failed to find the corresponding gene name.
Failed to find the corresponding gene name.
Failed to find the corresponding gene name.
Failed to find the corresponding gene name.
Failed to find the corresponding gene name.
Failed to find the corresponding gene name.
Failed to find the corresponding gene name.
1912. 221421_s_at: ADAMTS12
Failed to find the corresponding gene name.
Failed to find the corresponding gene name.
Failed to find the corresponding gene name.
Fail

Failed to find the corresponding gene name.
Failed to find the corresponding gene name.
Failed to find the corresponding gene name.
Failed to find the corresponding gene name.
Failed to find the corresponding gene name.
Failed to find the corresponding gene name.
Failed to find the corresponding gene name.
Failed to find the corresponding gene name.
Failed to find the corresponding gene name.
1944. 221800_s_at: FAAP100
Failed to find the corresponding gene name.
Failed to find the corresponding gene name.
Failed to find the corresponding gene name.
Failed to find the corresponding gene name.
Failed to find the corresponding gene name.
Failed to find the corresponding gene name.
Failed to find the corresponding gene name.
Failed to find the corresponding gene name.
Failed to find the corresponding gene name.
Failed to find the corresponding gene name.
Failed to find the corresponding gene name.
Failed to find the corresponding gene name.
1945. 221813_at: FBXO42
Failed to find the corres

Failed to find the corresponding gene name.
Failed to find the corresponding gene name.
Failed to find the corresponding gene name.
Failed to find the corresponding gene name.
Failed to find the corresponding gene name.
Failed to find the corresponding gene name.
Failed to find the corresponding gene name.
Failed to find the corresponding gene name.
Failed to find the corresponding gene name.
Failed to find the corresponding gene name.
Failed to find the corresponding gene name.
Failed to find the corresponding gene name.
Failed to find the corresponding gene name.
Failed to find the corresponding gene name.
Failed to find the corresponding gene name.
Failed to find the corresponding gene name.
Failed to find the corresponding gene name.
Failed to find the corresponding gene name.
Failed to find the corresponding gene name.
Failed to find the corresponding gene name.
Failed to find the corresponding gene name.
Failed to find the corresponding gene name.
Failed to find the corresponding

Failed to find the corresponding gene name.
1991. 38447_at: GRK2
Failed to find the corresponding gene name.
Failed to find the corresponding gene name.
Failed to find the corresponding gene name.
Failed to find the corresponding gene name.
Failed to find the corresponding gene name.
1992. 38707_r_at: E2F4
Failed to find the corresponding gene name.
Failed to find the corresponding gene name.
Failed to find the corresponding gene name.
Failed to find the corresponding gene name.
Failed to find the corresponding gene name.
Failed to find the corresponding gene name.
Failed to find the corresponding gene name.
Failed to find the corresponding gene name.
Failed to find the corresponding gene name.
Failed to find the corresponding gene name.
Failed to find the corresponding gene name.
Failed to find the corresponding gene name.
Failed to find the corresponding gene name.
Failed to find the corresponding gene name.
Failed to find the corresponding gene name.
Failed to find the corresponding

### Gene name matching

Using the POST request we can query online database to retrieve the corresponding gene names. The code below show an example of retrieving gene name for a particular gene ids. The complete version of the gene names matching was implemented in a separate IPython notebook as due to nature of the method the time to retrieve all the correspondences is long.

In [146]:
#Load necessary libraries
import requests
import json

In [264]:
headers = {'content-type': 'application/x-www-form-urlencoded'}
#params = 'q=reporter:204252_at'
params = 'q=1017,1018,reporter:204252_at&fields=name,symbol'
res = requests.post('http://mygene.info/v3/query', data=params, headers=headers)
response = json.loads(res.content)
print(response)

[{'query': '1017', '_id': '1017', '_score': 1.55, 'name': 'cyclin dependent kinase 2', 'symbol': 'CDK2'}, {'query': '1018', '_id': '1018', '_score': 1.55, 'name': 'cyclin dependent kinase 3', 'symbol': 'CDK3'}, {'query': 'reporter:204252_at', 'notfound': True}]


In [13]:
#Load the conversion table between affymetrix and common gene names
conversion_table = pd.read_csv('datasets/conversion_table.txt', delimiter='\t')


In [14]:
conversion_table

,id,affymetrix_id,gene_name
0,0,1007_s_at,DDR1
1,1,1053_at,RFC2
2,2,117_at,HSPA6
3,3,121_at,PAX8
4,4,1255_g_at,GUCA1A
5,5,1294_at,MIR5193
6,6,1316_at,THRA
7,7,1320_at,PTPN21
8,8,1405_i_at,CCL5
9,9,1431_at,CYP2E1


### Pathway retrieval

The code below is used to find pathways for a given gene. The pathways consists of genes that are interconnected. We create a list of all the pairs of genes that are connected in the pathway network and input them into a file. This is input data format necesary for the PFSNet.

In [9]:
#Load necessary libraries
from bioservices.kegg import KEGG
import progressbar
k = KEGG()

In [356]:
#This is the example of the code for finding a corresponding pathways for a given gene. 
#Then, for each pathway we find all the genes that constitute of the pathway (network).
#We write to a file a list of all the edges of all the pathway network for a given gene.
pathway = k.get_pathway_by_gene("CDK2", "hsa")
pfsnet_file = open("pfsnet_input_test.txt","w+")
for ind in pathway:
    print("************************************************")
    print(str(ind) +": "+str(pathway[ind]))
    print("************************************************")
    network = k.parse_kgml_pathway(ind)
    entries = pd.DataFrame.from_dict(network['entries'])
    for item in network['relations']:
        gene1 = entries[entries['id']==item['entry1']]
        gene2 = entries[entries['id']==item['entry2']]
        if gene1['gene_names'].values != None and gene2['gene_names'].values != None:
            gene1 = gene1['gene_names'].values[0].split(','[0])[0]
            gene2 = gene2['gene_names'].values[0].split(','[0])[0]
            pfsnet_file.write(str(pathway[ind]) +"\t"+ str(gene1)+"\t"+ str(gene2) + "\n")
            print(str(gene1) +" -> "+str(gene2)+"\n")
pfsnet_file.close()


************************************************
hsa04068: FoxO signaling pathway
************************************************
TGFBR1 -> SMAD2

SMAD4 -> SMAD2

C00008 -> STK11

C00020 -> STK11

STK11 -> PRKAG2

STK11 -> PRKAG2

STK11 -> FOXO6...

STK11 -> FOXO6...

TGFB1 -> TGFBR1

PRKAG2 -> FOXO6...

PRKAG2 -> FOXO6...

IGF1 -> IGF1R

INS -> INSR

IGF1R -> IRS1

INSR -> IRS1

IRS1 -> PIK3CA

PIK3CA -> C05981

C05981 -> PDPK1

PDPK1 -> CHUK

PDPK1 -> CHUK

PDPK1 -> AKT3

PDPK1 -> AKT3

PDPK1 -> C8orf44-SGK3...

PDPK1 -> C8orf44-SGK3...

AKT3 -> CHUK

AKT3 -> CHUK

CHUK -> FOXO6...

CHUK -> FOXO6...

AKT3 -> FOXO6...

AKT3 -> FOXO6...

C8orf44-SGK3... -> FOXO6...

C8orf44-SGK3... -> FOXO6...

NLK -> FOXO6...

NLK -> FOXO6...

CDK2 -> FOXO6...

CDK2 -> FOXO6...

TPTEP2-CSNK1E -> FOXO6...

TPTEP2-CSNK1E -> FOXO6...

MAPK14 -> FOXO6...

MAPK14 -> FOXO6...

C00025 -> GRM1

EGF -> EGFR

EGFR -> GRB2

GRB2 -> SOS1

SOS1 -> HRAS

HRAS -> ARAF

ARAF -> MAP2K1

ARAF -> MAP2K1

MAP2K1 -> MAPK


PPP1CA -> RB1

TP53 -> GADD45G

SMAD2 -> CDKN2B

ETS1 -> CDKN2A

TP53 -> CDKN1A

GATA4 -> TRAF3IP2

GATA4 -> IL1A

FOXO3 -> CDKN1A

TP53 -> CDKN1A

E2F1 -> MYBL2

E2F1 -> FOXM1

C07909 -> MTOR

HLA-A -> KIR2DL1

FOXO1 -> SMAD2

CDK4 -> CCND1

CDK2 -> CCNE1

CDK4 -> CCND1

CDK2 -> CCNE1

CDK2 -> CCNA2

CDK1 -> CCNB3...

CDK2 -> CCNE1

CDK4 -> CCND1

MYBL2 -> LIN54

FOXM1 -> LIN54

SLC25A4 -> VDAC1

SLC25A4 -> PPID

MRE11 -> RAD50

RAD50 -> NBN

MRE11 -> NBN

HUS1 -> RAD1

RAD9B... -> RAD1

RAD9B... -> HUS1

TRAF3IP2 -> NFKB1

TRAF3IP2 -> NFKB1

IL1A -> NFKB1

IL1A -> NFKB1

MTOR -> TP53

MTOR -> TP53

PIK3CA -> FOXO1

PIK3CA -> FOXO1

SQSTM1 -> GATA4

RASSF5 -> PPP1CA

SIRT1 -> TP53

RRAS2 -> CDKN2A

C16844... -> MAP2K3

HIPK3 -> TP53

HIPK3 -> TP53

ATM -> CHEK2

ATM -> CHEK2

************************************************
hsa04914: Progesterone-mediated oocyte maturation
************************************************
ADCY1 -> PRKACA

PIK3CA -> AKT3

CCNB1 -> CDK1

PGR -> PIK3CA




GNAS -> C00575

GNAS -> C00575

CTNNB1 -> TERT

TERT -> TCF7

AKT3 -> VEGFA

AKT3 -> VEGFA

RAF1 -> MAP2K1

RAF1 -> MAP2K1

MAP2K1 -> MAPK1

MAP2K1 -> MAPK1

PTGS2 -> C00584

STAT1 -> STAT2

STAT2 -> STAT1

PSMC1 -> RB1

UBE3A -> NFX1

NFX1 -> TERT

************************************************
hsa05166: Human T-cell leukemia virus 1 infection
************************************************
ITGAL -> ICAM1

ICAM1 -> ITGAL

B2M -> HLA-A

ICAM1 -> HRAS

HRAS -> MAP2K1

MAP2K1 -> MAPK1

IL2RB -> JAK1

PIK3CA -> C05981

C05981 -> AKT3

DLG1 -> PTEN

IKBKG -> CHUK

CHUK -> NFKB2

ADCY1 -> C00575

C00575 -> PRKACA

AKT3 -> JUN

AKT3 -> JUN

TNFRSF1A -> MAP3K1

TNFRSF1A -> MAP3K1

TNFRSF1A -> MAP3K3

TNFRSF1A -> MAP3K3

IL1R1 -> MAP3K3

IL1R1 -> MAP3K3

CD3D -> MAP3K3

CD3D -> MAP3K3

LTBR -> MAP3K14

LTBR -> MAP3K14

CD40 -> MAP3K14

CD40 -> MAP3K14

TNFRSF13C -> MAP3K14

TNFRSF13C -> MAP3K14

TGFB1 -> TGFBR1

MYC -> CCND2

NFATC1 -> IL2

STAT5A -> IL2

NFYB -> HLA-DMA

CREB3 -> CCND1

T


AGT -> LPAR6

AGT -> CXCR4

C00681 -> LPAR6

C00681 -> CXCR4

CXCL12 -> CXCR4

C00584 -> PTGER1

************************************************
hsa05203: Viral carcinogenesis
************************************************
MDM2 -> TP53

CREBBP -> TP53

PSMC1 -> RB1

************************************************
hsa05215: Prostate cancer
************************************************
PIK3CA -> PDPK1

PIK3CA -> PDPK1

PIK3CA -> AKT3

PIK3CA -> AKT3

EGF -> EGFR

EGFR -> PIK3CA

EGFR -> GRB2

GRB2 -> SOS1

SOS1 -> HRAS

HRAS -> ARAF

ARAF -> MAP2K1

ARAF -> MAP2K1

MAP2K1 -> MAPK1

MAP2K1 -> MAPK1

PDPK1 -> AKT3

PDPK1 -> AKT3

AKT3 -> MDM2

AKT3 -> MDM2

AKT3 -> CHUK

AKT3 -> CHUK

AKT3 -> MTOR

AKT3 -> MTOR

AKT3 -> CASP9

AKT3 -> CASP9

AKT3 -> BAD

AKT3 -> BAD

AKT3 -> FOXO1

AKT3 -> FOXO1

AKT3 -> CDKN1A

AKT3 -> CDKN1A

AKT3 -> CDKN1B

AKT3 -> CDKN1B

AKT3 -> GSK3B

AKT3 -> GSK3B

GSK3B -> CREB3

GSK3B -> CREB3

GSK3B -> CTNNB1

GSK3B -> CTNNB1

CHUK -> NFKBIA

CHUK -> NFKB

In [72]:
#Load the conversion table between affymetrix and common gene names
pathways = pd.read_csv('pathways.txt', delimiter='\t')


In [73]:
pathways

,id,pathway_id,pathway_name
0,0,hsa03030,DNA replication
1,1,hsa03420,Nucleotide excision repair
2,2,hsa03430,Mismatch repair
3,3,hsa03040,Spliceosome
4,4,hsa04010,MAPK signaling pathway
5,5,hsa04141,Protein processing in endoplasmic reticulum
6,6,hsa04144,Endocytosis
7,7,hsa04213,Longevity regulating pathway - multiple species
8,8,hsa04612,Antigen processing and presentation
9,9,hsa04915,Estrogen signaling pathway


In [402]:
#For all the distinct pathways find all the genes that build a given pathway. 
#Create a file with a list of all the interrelations between the genes in a given pathway.
pfsnet_file = open("pfsnet_pathways.txt","w+")
pfsnet_file.write("pathway\tgene1\tgene2\n")
for ind in pathways:
    network = k.parse_kgml_pathway(ind)
    entries = pd.DataFrame.from_dict(network['entries'])
    for item in network['relations']:
        gene1 = entries[entries['id']==item['entry1']]
        gene2 = entries[entries['id']==item['entry2']]
        if gene1['gene_names'].values != None and gene2['gene_names'].values != None:
            gene1 = gene1['gene_names'].values[0].split(','[0])[0]
            gene2 = gene2['gene_names'].values[0].split(','[0])[0]
            pfsnet_file.write(str(pathway[ind]) +"\t"+ str(gene1)+"\t"+ str(gene2) + "\n")
pfsnet_file.close()


### Relapse/non-relapse data split

In [20]:
relapse = pd.DataFrame()
non_relapse = pd.DataFrame()
relapse['ID'] = conversion_table['gene_name']
non_relapse['ID'] = conversion_table['gene_name']
for index in relapse_dataset[relapse_dataset['relapse (1=True)'] == 1].index:
    patient_id = relapse_dataset.iloc[index]['asscession number']
    relapse[patient_id] = dataset[patient_id]

for index in relapse_dataset[relapse_dataset['relapse (1=True)'] == 0].index:
    patient_id = relapse_dataset.iloc[index]['asscession number']
    non_relapse[patient_id] = dataset[patient_id]


In [73]:
relapse = relapse.drop(relapse[relapse['ID'] == '-'].index)

columns = relapse.columns[1:]
temp_array=[]

unique_gene_id = relapse['ID'].unique()
for gene_id in unique_gene_id:
    temp_dataframe = relapse[relapse['ID']==gene_id]
    row_entry = [gene_id]
    for column in columns:
        row_entry.append(temp_dataframe[column].max())
    temp_array.append(row_entry)
    
relapse_cleaned = pd.DataFrame(temp_array, columns = relapse.columns)  
relapse_cleaned.to_csv("pfsnet_relapse_cleaned.txt", sep='\t')

In [21]:
relapse_cleaned = pd.read_csv('pfsnet_relapse_cleaned.txt', delimiter='\t')
relapse_cleaned

,ID,GENE_NAME,GSM36797,GSM36800,GSM36835,GSM36838,GSM36839,GSM36858,GSM36860,GSM36862,...,GSM36983,GSM36985,GSM36986,GSM37005,GSM37006,GSM37007,GSM37008,GSM37011,GSM37013,GSM36911
0,0,DDR1,3510.9,6242.9,3316.9,3819.4,5548.5,4042.5,2891.2,5679.4,...,4173.9,3445.9,3997.7,5072.2,3974.0,3545.4,2656.1,9425.9,2926.3,5486.6
1,1,RFC2,391.3,424.7,435.2,386.0,260.0,374.3,382.5,611.9,...,367.0,329.1,351.4,371.0,291.6,399.1,337.0,446.2,433.5,357.8
2,2,HSPA6,582.6,861.1,438.2,479.1,342.7,562.3,446.6,973.1,...,242.3,305.0,1868.8,650.9,754.6,773.7,631.6,518.9,330.4,1173.0
3,3,PAX8,1896.1,1170.6,2478.2,1749.7,1782.3,1431.9,1289.0,1722.0,...,1140.4,859.8,1153.9,1317.1,1898.3,1273.1,1137.8,1264.4,1912.2,1055.1
4,4,GUCA1A,114.3,135.1,263.0,153.9,185.6,112.5,112.2,117.1,...,141.2,123.0,121.5,160.4,265.4,101.6,150.3,73.9,133.8,62.6
5,5,MIR5193,686.4,767.1,627.1,690.2,647.7,385.0,1112.3,729.8,...,937.7,666.3,871.4,426.5,401.0,966.0,1013.3,802.4,749.8,740.5
6,6,THRA,1429.7,1124.4,1550.8,1508.6,1446.4,1663.9,677.5,1159.2,...,1679.8,747.7,549.4,1144.8,1276.4,994.9,1264.2,765.9,1259.9,669.6
7,7,PTPN21,102.4,17.4,191.3,136.6,86.7,80.7,77.7,184.2,...,123.8,43.2,51.5,96.1,81.6,17.1,99.5,34.2,31.1,113.6
8,8,CCL5,707.1,412.1,900.6,436.7,305.2,475.6,4406.8,1421.3,...,1587.0,2258.1,510.4,378.7,569.0,1162.7,631.9,814.1,698.9,389.5
9,9,CYP2E1,76.8,267.1,138.0,365.4,315.6,213.5,270.0,51.1,...,82.1,132.1,175.6,162.0,428.1,55.4,140.3,160.6,239.6,156.7


In [74]:
non_relapse = non_relapse.drop(non_relapse[non_relapse['ID'] == '-'].index)

columns = non_relapse.columns[1:]
temp_array=[]

unique_gene_id = non_relapse['ID'].unique()
for gene_id in unique_gene_id:
    temp_dataframe = non_relapse[non_relapse['ID']==gene_id]
    row_entry = [gene_id]
    for column in columns:
        row_entry.append(temp_dataframe[column].max())
    temp_array.append(row_entry)
    
non_relapse_cleaned = pd.DataFrame(temp_array, columns = non_relapse.columns)  
non_relapse_cleaned.to_csv("pfsnet_non_relapse_cleaned.txt", sep='\t')



In [20]:
non_relapse_cleaned = pd.read_csv('pfsnet_non_relapse_cleaned.txt', delimiter='\t')
non_relapse_cleaned

,ID,GENE_NAME,GSM36793,GSM36796,GSM36798,GSM36801,GSM36834,GSM36836,GSM36837,GSM36855,...,GSM37032,GSM37033,GSM37043,GSM37044,GSM37045,GSM37046,GSM36887,GSM36889,GSM36892,GSM36894
0,0,DDR1,3998.2,3562.3,2974.8,4866.6,5034.9,3055.5,5727.4,5458.0,...,3027.6,3512.7,3480.5,4602.0,2346.0,2185.7,3883.4,1841.7,2147.5,2127.6
1,1,RFC2,238.1,592.4,391.3,333.5,693.9,234.8,345.5,292.9,...,454.1,279.2,416.4,506.9,440.8,353.6,271.1,375.1,150.6,354.1
2,2,HSPA6,329.2,304.6,1231.2,370.8,761.0,428.3,295.5,2386.3,...,615.2,430.2,728.7,583.4,487.5,291.5,510.9,841.9,909.7,333.2
3,3,PAX8,1982.6,1625.7,1777.3,1893.3,1170.5,1567.8,1623.2,1577.7,...,1324.8,1384.9,878.6,1241.8,798.8,1082.0,984.2,1087.1,778.8,1083.7
4,4,GUCA1A,187.9,115.7,110.0,152.8,124.3,160.6,177.7,201.1,...,175.2,145.1,130.7,72.3,170.4,120.7,163.5,180.0,45.9,93.5
5,5,MIR5193,793.2,768.5,702.8,1236.5,677.6,380.7,1379.5,482.6,...,917.4,735.6,565.1,770.7,707.2,575.9,914.4,669.2,865.6,820.3
6,6,THRA,1062.5,1023.4,1509.6,1217.1,3416.5,1413.7,680.0,1403.7,...,905.4,1226.6,900.4,665.4,439.7,1306.8,1052.6,1004.5,992.8,665.2
7,7,PTPN21,96.1,109.3,84.5,47.1,28.3,62.5,87.8,88.1,...,58.6,98.0,86.4,100.7,56.3,50.4,119.6,51.3,47.1,90.2
8,8,CCL5,828.4,1055.5,3255.1,9906.9,1036.7,594.3,1769.7,657.4,...,1881.0,2524.7,797.9,559.5,11570.9,2104.8,1037.9,2683.1,3623.5,1841.8
9,9,CYP2E1,311.4,200.2,173.9,100.2,167.9,230.0,192.5,124.1,...,198.7,113.4,128.3,141.1,73.2,146.8,296.6,259.4,319.0,123.2


In [32]:
pfsnet_pathways_cleaned = pd.read_csv('pfsnet_pathways_cleaned.txt', delimiter='\t')
pfsnet_pathways_cleaned

,Unnamed: 0,pathway,gene1,gene2
0,0,hsa04010,GNA12,RASA2
1,1,hsa04010,RAPGEF2,RRAS2
2,2,hsa04010,RASGRF1,RRAS2
3,3,hsa04010,RASGRP1,RRAS2
4,4,hsa04010,RAP1A,BRAF
5,5,hsa04010,TNF,TNFRSF1A
6,6,hsa04010,IL1A,IL1R1
7,7,hsa04010,FASLG,FAS
8,8,hsa04010,TGFB1,TGFBR1
9,9,hsa04010,FAS,DAXX


In [33]:
genes_list = conversion_table.drop_duplicates(['gene_name'])['gene_name']
genes_list.values

array(['DDR1', 'RFC2', 'HSPA6', ..., 'FAM86B1', 'SNHG17', 'LOC100505915'],
      dtype=object)

In [ ]:
gene_list = pd.concat([pfsnet_pathways_cleaned['gene1'], pfsnet_pathways_cleaned['gene2']])
gene_list = pd.DataFrame(gene_list, columns=['gene_name'])
gene_list = gene_list.drop_duplicates(['gene_name'])
count = 0
for gene in gene_list['gene_name']:
    if gene in genes_list.values:
        count = count + 1
    else:
        df_temp = pd.DataFrame([np.concatenate(([non_relapse_cleaned.index.size+1, gene], np.zeros(non_relapse_cleaned.columns.size-2)), axis=0)], columns = non_relapse_cleaned.columns)
        
        
        df_temp_2 = pd.DataFrame([np.concatenate(([relapse_cleaned.index.size+1, gene], np.zeros(relapse_cleaned.columns.size-2)), axis=0)], columns = relapse_cleaned.columns)

        non_relapse_cleaned = non_relapse_cleaned.append(df_temp, ignore_index=True)
        relapse_cleaned = relapse_cleaned.append(df_temp_2, ignore_index=True)
        
print(count)
print(gene_list.size)
print(genes_list.size)

In [ ]:
non_relapse_cleaned

In [ ]:
relapse_cleaned

In [ ]:
non_relapse_cleaned.to_csv("pfsnet_non_relapse_cleaned.txt", sep='\t')
relapse_cleaned.to_csv("pfsnet_relapse_cleaned.txt", sep='\t')

## 2. PFSNet score generation

## 3. Classification model prediction